## Preprocessing I Image stats
( overwrites image_features_nb.py )


* estimate intensity stats in red / green channel
* calculate nuc and syn stats for each image
* save for future steps



In [16]:
# local imports
from pathlib import Path
# from os.path import isfile
import numpy as np
import pandas as pd

from src.data.read_data import get_image_name
from src.features.build_features import get_img_percentiles

%load_ext autoreload
%autoreload 2

### Image stats
This code takes in raw tif or tiff file ... does stuff to it and it's all good now.

In [8]:
# %% path with all the images
pallium_path = Path("D:/Code/repos/psd95_segmentation/data/raw/img/pallium")

# get list of tif files in the folder
files = [x for x in pallium_path.glob('*tif*') if x.is_file()]
print(files[0])
len(files)

D:\Code\repos\psd95_segmentation\data\raw\img\pallium\Image_1-01H6.ome.tif


88

In [9]:
# Load a csv that has all the info for the images, segmentation , etc

info_df = pd.read_csv('D:/Code/repos/psd95_segmentation/data/raw/karls_good2.csv')

info_year = info_df.loc[:, ['Source Image', 'Subject Issue Date','Subject']].sort_values(by=['Source Image'])
info_year.drop_duplicates(subset='Source Image', inplace=True)
info_year

,Source Image,Subject Issue Date
619,1-018J,6/1/2017
620,1-018M,6/1/2017
378,1-0196,9/9/2016
56,1-01AA,7/4/2017
53,1-01AG,7/4/2017
...,...,...
1125,1-1RSG,NaN
1136,1-1RSM,NaN
1123,1-1RSP,NaN
1118,1-1RTG,NaN


Here we're finding in that big table from Karl all the images that we have downloaded. Getting theit names and years:

In [10]:
# Match the files you have with the years, extract file names
years = np.zeros((len(files), 1))
names = []
for count, file_name in enumerate(files):
    img_name = get_image_name(file_name)
    year = info_year.loc[info_year['Source Image'] == img_name,
                         'Subject Issue Date'].values[0].split(sep='/')[-1]
    print(f"#{count} image {img_name} year {year}")
    years[count] = year
    names.append(img_name)

#0 image 1-01H6 year 2017
#1 image 1-01QC year 2017
#2 image 1-01QP year 2017
#3 image 1-01RJ year 2017
#4 image 1-01RM year 2017
#5 image 1-020C year 2017
#6 image 1-020G year 2017
#7 image 1-02K6 year 2016
#8 image 1-02Q6 year 2016
#9 image 1-039E year 2017
#10 image 1-039M year 2017
#11 image 1-03G4 year 2017
#12 image 1-0HN6 year 2017
#13 image 1-0HNJ year 2017
#14 image 1-0HPY year 2017
#15 image 1-0HQA year 2017
#16 image 1-0HXP year 2017
#17 image 1-0HY2 year 2017
#18 image 1-0R4Y year 2017
#19 image 1-0R5A year 2017
#20 image 1-0RXP year 2018
#21 image 1-0RY2 year 2018
#22 image 1-0RYE year 2018
#23 image 1-0RYT year 2018
#24 image 1-0S0P year 2018
#25 image 1-0S12 year 2018
#26 image 1-0V1M year 2018
#27 image 1-0V20 year 2018
#28 image 1-0VA0 year 2018
#29 image 1-0VAC year 2018
#30 image 1-0VCR year 2018
#31 image 1-0VD4 year 2018
#32 image 1-0VDG year 2018
#33 image 1-0VDW year 2018
#34 image 1-0VF8 year 2018
#35 image 1-0VFM year 2018
#36 image 1-0Z30 year 2018
#37 image 1

In [11]:
# Get the intensity precentiles for green and red channels.

# use padding, since you don't want the 0 boarders to effect the stats
padding = [[0, 0], [50, 500], [20, 20]]

# loads a file, calculates percentiles
prc_green = get_img_percentiles("green", files, padding)
prc_red = get_img_percentiles("red", files, padding)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [12]:
# merge year w percentile

img_intensity_df = pd.DataFrame(np.concatenate((years, prc_green, prc_red), axis=1),
                                columns=['Year', 'green_prc_0', 'green_prc_10',
                                         'green_prc_20', 'green_prc_30',
                                         'green_prc_40', 'green_prc_50',
                                         'green_prc_60', 'green_prc_70',
                                         'green_prc_80', 'green_prc_90',
                                         'green_prc_100',
                                         'red_prc_0', 'red_prc_10',
                                         'red_prc_20', 'red_prc_30',
                                         'red_prc_40', 'red_prc_50',
                                         'red_prc_60', 'red_prc_70',
                                         'red_prc_80', 'red_prc_90',
                                         'red_prc_100'])
img_intensity_df.insert(0, 'Name', names, True)

In [13]:
# Figure out what time point it is.
""" 
If the old image name ends with a 6 or 5 then its tp 2; 3 or 2 is tp 1.
"""
# %%

id_segments = []
time_point = []
for count, img_name in enumerate(names):

    id_segment = info_df.loc[info_df['Source Image'] == img_name, 'ID'].values[0]
    print(f"#{count} image {img_name} ID {id_segment}")
    id_segments.append(id_segment)

    if id_segment[-2] == '6' or id_segment[-2] == '5':
        time_point.append(2)
    elif id_segment[-2] == '3' or id_segment[-2] == '2':
        time_point.append(1)
    else:
        raise ValueError("Don't know what time point it is")

# add to the data frame
img_intensity_df.insert(1, 'tp', time_point, True)

#0 image 1-01H6 ID SynImgZfZdu20170516A3B
#1 image 1-01QC ID SynImgZfDsy20170316A3B
#2 image 1-01QP ID SynImgZfDsy20170316B6B
#3 image 1-01RJ ID SynImgZfDsy20170316A6B
#4 image 1-01RM ID SynImgZfDsy20170316B3C
#5 image 1-020C ID SynImgZfDsy20170329C3B
#6 image 1-020G ID SynImgZfDsy20170329C6B
#7 image 1-02K6 ID SynImgZfDsy20161202B3C
#8 image 1-02Q6 ID SynImgZfDsy20161202B6C
#9 image 1-039E ID SynImgZfDsy20170412A3B
#10 image 1-039M ID SynImgZfDsy20170412A6B
#11 image 1-03G4 ID SynImgZfZdu20170516A6B
#12 image 1-0HN6 ID SynImgZfZdu20171114B3A
#13 image 1-0HNJ ID SynImgZfZdu20171114B6A
#14 image 1-0HPY ID SynImgZfZdu20171121A3A
#15 image 1-0HQA ID SynImgZfZdu20171121A6A
#16 image 1-0HXP ID SynImgZfDsy20171019A3A
#17 image 1-0HY2 ID SynImgZfDsy20171019A6A
#18 image 1-0R4Y ID SynImgZfDsy20171222B3A
#19 image 1-0R5A ID SynImgZfDsy20171222B6A
#20 image 1-0RXP ID SynImgZfZdu20180116A3A
#21 image 1-0RY2 ID SynImgZfZdu20180116A6A
#22 image 1-0RYE ID SynImgZfZdu20180116B3A
#23 image 1-0RYT ID S

### Nuclear stats
Calculates percentiles of nuclear intensity. 


If no nuclear csv is present, sets all percentiles for that image to ZERO.

In [17]:
# Calculate and add nuclear percentiles to the data frame
nuc_path = 'D:/Code/repos/psd95_segmentation/data/raw/csv/pallium/nuc/'
# %%

nuc_prc = np.zeros((len(names), 5))

# for each image , read the corresponding csv and calculate pct
for count, img_name in enumerate(names):
    print(f"#{count} image {img_name}")
    csv_file = nuc_path + img_name + '_nuclei_only.csv'
    
    if Path(csv_file).exists():
        nuc_csv_df = pd.read_csv(csv_file, skiprows=[1])

        nuc_intensity = nuc_csv_df.loc[:, ['raw core']]
        nuc_prc[count, :] = np.percentile(nuc_intensity, [10, 25, 50, 75, 90])
    else:
        print("Skipped")

nuc_prc_df = pd.DataFrame(nuc_prc, columns=['nuc_prc_10', 'nuc_prc_25',
                                            'nuc_prc_50', 'nuc_prc_75', 'nuc_prc_90'])
# add to the main df
img_intensity_df = pd.concat([img_intensity_df, nuc_prc_df], axis=1)

#0 image 1-01H6
#1 image 1-01QC
Skipped
#2 image 1-01QP
Skipped
#3 image 1-01RJ
Skipped
#4 image 1-01RM
Skipped
#5 image 1-020C
Skipped
#6 image 1-020G
Skipped
#7 image 1-02K6
Skipped
#8 image 1-02Q6
Skipped
#9 image 1-039E
Skipped
#10 image 1-039M
Skipped
#11 image 1-03G4
#12 image 1-0HN6
#13 image 1-0HNJ
#14 image 1-0HPY
#15 image 1-0HQA
#16 image 1-0HXP
#17 image 1-0HY2
#18 image 1-0R4Y
#19 image 1-0R5A
#20 image 1-0RXP
#21 image 1-0RY2
#22 image 1-0RYE
#23 image 1-0RYT
#24 image 1-0S0P
#25 image 1-0S12
#26 image 1-0V1M
#27 image 1-0V20
#28 image 1-0VA0
#29 image 1-0VAC
#30 image 1-0VCR
#31 image 1-0VD4
#32 image 1-0VDG
#33 image 1-0VDW
#34 image 1-0VF8
#35 image 1-0VFM
#36 image 1-0Z30
#37 image 1-0Z3C
#38 image 1-0Z3R
#39 image 1-0Z44
#40 image 1-0Z4G
#41 image 1-0Z4W
#42 image 1-0Z60
#43 image 1-0Z6C
#44 image 1-0Z8R
#45 image 1-0Z94
#46 image 1-130G
#47 image 1-130W
#48 image 1-13H6
#49 image 1-13HC
#50 image 1-13RA
#51 image 1-13RG
#52 image 1-14P0
#53 image 1-14P6
#54 image 1-

### Synapse stats

In [18]:
# Calculate and add synapse percentiles to the data frame
syn_path = 'D:/Code/repos/psd95_segmentation/data/raw/csv/pallium/syn/'

syn_prc = np.zeros((len(names), 5))

for count, img_name in enumerate(names):
    print(f"#{count} image {img_name}")
    done = 0
    segmentations = info_df.loc[info_df['Source Image'] == img_name,
                                ['RID', 'classifier_name', 'Segments Filtered URL']]
    
    # there will be multiple entries for each image ...
    for index, segmentation in segmentations.iterrows():
        rid = segmentation[0]
        cl_name = segmentation[1]
        # need url to figure out type ...
        url = segmentation[2]
        csv_type = url.split('.')[1]
        
        # there are nuclear ans synaptic files, you want synaptic 
        if csv_type == 'synapses_only':
            csv_file = syn_path + img_name + '_' + rid + '_' + csv_type + '.csv'

            # grab the first segmentation you have for this image
            if Path(csv_file).exists() and done == 0:
                done = 1

                syn_csv_df = pd.read_csv(csv_file, skiprows=[1])
                syn_intensity = syn_csv_df.loc[:, ['raw core']]
                syn_prc[count, :] = np.percentile(syn_intensity,
                                                  [10, 25, 50, 75, 90])
    # if you ran through all entries but didn't have 'synapses_only'
    if done==0:
        print("Skipped")

syn_prc_df = pd.DataFrame(syn_prc, columns=['syn_prc_10', 'syn_prc_25',
                                            'syn_prc_50', 'syn_prc_75', 'syn_prc_90'])

img_intensity_df = pd.concat([img_intensity_df, syn_prc_df], axis=1)

#0 image 1-01H6
#1 image 1-01QC
Skipped
#2 image 1-01QP
Skipped
#3 image 1-01RJ
Skipped
#4 image 1-01RM
Skipped
#5 image 1-020C
Skipped
#6 image 1-020G
Skipped
#7 image 1-02K6
Skipped
#8 image 1-02Q6
Skipped
#9 image 1-039E
Skipped
#10 image 1-039M
Skipped
#11 image 1-03G4
#12 image 1-0HN6
#13 image 1-0HNJ
#14 image 1-0HPY
#15 image 1-0HQA
#16 image 1-0HXP
#17 image 1-0HY2
#18 image 1-0R4Y
#19 image 1-0R5A
#20 image 1-0RXP
#21 image 1-0RY2
#22 image 1-0RYE
#23 image 1-0RYT
#24 image 1-0S0P
#25 image 1-0S12
#26 image 1-0V1M
#27 image 1-0V20
#28 image 1-0VA0
#29 image 1-0VAC
#30 image 1-0VCR
#31 image 1-0VD4
#32 image 1-0VDG
#33 image 1-0VDW
#34 image 1-0VF8
#35 image 1-0VFM
#36 image 1-0Z30
#37 image 1-0Z3C
#38 image 1-0Z3R
#39 image 1-0Z44
#40 image 1-0Z4G
#41 image 1-0Z4W
#42 image 1-0Z60
#43 image 1-0Z6C
#44 image 1-0Z8R
#45 image 1-0Z94
#46 image 1-130G
#47 image 1-130W
#48 image 1-13H6
#49 image 1-13HC
#50 image 1-13RA
#51 image 1-13RG
#52 image 1-14P0
#53 image 1-14P6
#54 image 1-

### Save data frame as csv

In [19]:
img_intensity_df

,Name,tp,Year,green_prc_0,green_prc_10,green_prc_20,green_prc_30,green_prc_40,green_prc_50,green_prc_60,...,nuc_prc_10,nuc_prc_25,nuc_prc_50,nuc_prc_75,nuc_prc_90,syn_prc_10,syn_prc_25,syn_prc_50,syn_prc_75,syn_prc_90
0,1-01H6,1,2017.0,5.0,112.0,119.0,126.0,135.0,148.0,177.0,...,972.5091,1453.12250,2120.3000,4634.37000,6109.0670,360.383400,394.595000,449.71000,527.465000,619.616600
1,1-01QC,1,2017.0,0.0,106.0,110.0,116.0,125.0,146.0,187.0,...,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000
2,1-01QP,2,2017.0,0.0,105.0,109.0,113.0,118.0,127.0,148.0,...,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000
3,1-01RJ,2,2017.0,0.0,107.0,113.0,120.0,133.0,164.0,212.0,...,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000
4,1-01RM,1,2017.0,0.0,105.0,108.0,112.0,117.0,126.0,144.0,...,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1-19CP,2,2019.0,2.0,109.0,114.0,119.0,126.0,137.0,153.0,...,189.0194,338.06325,1296.4650,4289.13000,5801.9320,203.424500,228.788500,262.50150,305.337000,354.234500
84,1-1D6M,1,2019.0,4.0,104.0,106.0,109.0,113.0,119.0,132.0,...,150.5914,155.44000,165.2815,631.66350,7416.9047,187.780456,198.977580,219.49329,247.177120,281.287570
85,1-1D6T,2,2019.0,0.0,105.0,108.0,112.0,118.0,128.0,140.0,...,156.3078,159.73575,166.2935,202.79150,581.6630,183.047670,192.293850,208.10617,230.618620,258.946800
86,1-1E8T,1,2019.0,0.0,106.0,109.0,114.0,119.0,129.0,148.0,...,163.2776,171.29650,186.2165,791.04425,1465.4000,203.018466,216.423975,237.79242,274.928455,331.131726


In [22]:
int_path = "D:/Code/repos/psd95_segmentation/data/processed/"
int_file = int_path + "Intensity_stats.csv"
img_intensity_df.to_csv(int_file) 